In [2]:
from tqdm.notebook import trange, tqdm
from utilities import files
import os.path as op
import numpy as np
from os import sep
import itertools as it
import json

In [3]:
def many_is_in(multiple, target):
    check_ = []
    for i in multiple:
        check_.append(i  in target)
    return any(check_)

def cat(options, target):
    for i in options:
        if i in target:
            return i

In [4]:
exclude=[
    "sub-002-ses-01",
    "sub-002-ses-02",
    "sub-006-ses-01",
    "sub-006-ses-02",
    "sub-008-ses-01"
]

In [14]:
ds_path = "/home/common/bonaiuto/cued_action_meg"
subs_path = op.join(ds_path, "derivatives", "processed")
sub_paths = files.get_folders_files(subs_path)[0]
sub_paths.sort()
subs = [i.split(sep)[-1] for i in sub_paths]
epochs = {sub: {"motor": [], "visual": []} for sub in subs}
for sub_path in sub_paths:
    sub = sub_path.split(sep)[-1]
    sessions_paths = files.get_folders_files(sub_path)[0]
    sessions_paths = [i for i in sessions_paths if "ses-" in i]
    session_motor = {i.split(sep)[-1]: files.get_files(i, "autoreject-sub", "-motor-epo.fif")[2] for i in sessions_paths}
    session_visual = {i.split(sep)[-1]: files.get_files(i, "autoreject-sub", "-visual1-epo.fif")[2] for i in sessions_paths}
    session_motor = list(it.chain(*[session_motor[i] for i in session_motor.keys()]))
    session_visual = list(it.chain(*[session_visual[i] for i in session_visual.keys()]))
    session_motor = [i for i in session_motor if not many_is_in(exclude, i)]
    session_motor.sort()
    session_visual = [i for i in session_visual if not many_is_in(exclude, i)]
    session_visual.sort()
    epochs[sub]["visual"] = session_visual
    epochs[sub]["motor"] = session_motor
    

In [15]:
epochs

{'sub-001': {'motor': ['/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/autoreject-sub-001-ses-01-001-motor-epo.fif',
   '/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/autoreject-sub-001-ses-01-002-motor-epo.fif',
   '/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/autoreject-sub-001-ses-01-003-motor-epo.fif'],
  'visual': ['/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/autoreject-sub-001-ses-01-001-visual1-epo.fif',
   '/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/autoreject-sub-001-ses-01-002-visual1-epo.fif',
   '/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-001/ses-01/autoreject-sub-001-ses-01-003-visual1-epo.fif']},
 'sub-002': {'motor': ['/home/common/bonaiuto/cued_action_meg/derivatives/processed/sub-002/ses-03/autoreject-sub-002-ses-03-001-motor-epo.fif',
   '/home/common/bonaiuto/cued_action_meg/derivatives/processed/su